In [3]:
import numpy as np
import pandas as pd
import time, json, random
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN, KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib
import json
print('matplotlib: {}'.format(matplotlib.__version__))


RANDOM_STATE = 3


matplotlib: 3.5.3


In [4]:
DS = pd.read_csv("./total.csv", sep=',', header=[0])
X_data = DS.drop(columns=['file', 'scope'])
X_data = X_data.drop_duplicates()
X_data = X_data.drop(columns=['proj', 'commit'])
X_data = X_data[X_data["UseTree"] == 0]
X_data = X_data[X_data["UseTree"] == 0]
X_data = X_data[X_data["ItemUse"] == 0]
X_data = X_data[X_data["UseGlob"] == 0]
X_data = X_data[X_data["UseList"] == 0]
X_data = X_data[X_data["UsePath"] == 0]
X_data = X_data[X_data["ItemForeignMod"] == 0]
X_data = X_data[X_data["ItemMod"] == 0]
print(len(X_data))

60583


In [5]:
dbscan_data = []
Z = [5,7,9]
E = [0.01, 0.001, 0.0001]
for z in Z:
    for e in E:
        db = DBSCAN(eps=e, min_samples=z).fit(X_data)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_
        dbscan_labels = pd.Series(labels, name="label")

        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)

        print("Epsilon", e)
        print("Z", z)
        print("Estimated number of clusters: %d" % n_clusters_)
        print("Estimated number of noise points: %d" % n_noise_)
        dbscan_data.append((z, e, n_clusters_, n_noise_))
print(dbscan_data)

Epsilon 0.01
Z 5
Estimated number of clusters: 331
Estimated number of noise points: 19902
Epsilon 0.001
Z 5
Estimated number of clusters: 556
Estimated number of noise points: 40944
Epsilon 0.0001
Z 5
Estimated number of clusters: 577
Estimated number of noise points: 50441
Epsilon 0.01
Z 7
Estimated number of clusters: 182
Estimated number of noise points: 20931
Epsilon 0.001
Z 7
Estimated number of clusters: 312
Estimated number of noise points: 42379
Epsilon 0.0001
Z 7
Estimated number of clusters: 330
Estimated number of noise points: 51852
Epsilon 0.01
Z 9
Estimated number of clusters: 128
Estimated number of noise points: 21535
Epsilon 0.001
Z 9
Estimated number of clusters: 226
Estimated number of noise points: 43189
Epsilon 0.0001
Z 9
Estimated number of clusters: 226
Estimated number of noise points: 52705
[(5, 0.01, 331, 19902), (5, 0.001, 556, 40944), (5, 0.0001, 577, 50441), (7, 0.01, 182, 20931), (7, 0.001, 312, 42379), (7, 0.0001, 330, 51852), (9, 0.01, 128, 21535), (9, 

In [6]:
Z = 5
e = 0.0001
db = DBSCAN(eps=e, min_samples=Z).fit(X_data)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
dbscan_labels = pd.Series(labels, name="label")

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Epsilon", e)
print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Epsilon 0.0001
Estimated number of clusters: 577
Estimated number of noise points: 50441


In [7]:
X_df = pd.DataFrame(data=X_data)
dbscan_labels = pd.Series(labels, name="label", index=X_df.index)
full_dbscan_data = pd.concat([X_df, dbscan_labels], axis = 1)
full_dbscan_clusters = []
for label in range(n_clusters_):
    cluster = full_dbscan_data[full_dbscan_data["label"] == label]
    projects = set()
    for index1, row in cluster.iterrows():
        projects.add(DS.loc[index1, 'proj'])
    if len(projects) > 1:
        full_dbscan_clusters.append(cluster)
print("Clusters with cross-project bugs:",len(full_dbscan_clusters))

Clusters with cross-project bugs: 428


In [8]:
## Sampling
full_dbscan_clusters.sort(key=len, reverse=True)
DPs = []
for ID in range(1, len(full_dbscan_clusters)):
    samples = full_dbscan_clusters[ID-1]
    projs = set()
    for index1, row in samples.iterrows():
        DPs.append(row)
m = 100
sample = None
for i in range(1000):
    random.seed(RANDOM_STATE+i)
    df = pd.DataFrame(DPs)
    df_sample = pd.DataFrame(random.sample(DPs, k=30))
    sample_error = abs((df.mean(axis = 0) - df_sample.mean(axis = 0)).mean())
    if m > sample_error:
        m = sample_error
        sample = df_sample
print(m)
for index1, row in sample.iterrows():
    print("https://github.com/"+DS.loc[index1, 'proj']+"/commit/"+DS.loc[index1, 'commit']+" "+DS.loc[index1, 'file']+" "+str(DS.loc[index1, 'scope']))

0.00017972645547393155
https://github.com/tokio-rs/tokio/commit/a8500632113a527780fb9c756e3d8b8a85d0f754 registration.rs 3-Impl
https://github.com/nushell/nushell/commit/8925ca5da3cc519b9a904fd95478a83a86a58990 external.rs nan
https://github.com/nushell/nushell/commit/2b37ae3e81b54e3128cc6e3f05f27525f6974082 external.rs 1-Fn
https://github.com/denoland/deno/commit/dccf5e0c5c7f04409809104dd23472bcc058e170 runtime.rs 4-Struct
https://github.com/swc-project/swc/commit/dba90eae875bbaec9911c6dd72d482b75de77982 tests.rs 54-Fn
https://github.com/yewstack/yew/commit/78d4204a9a52a826bd2be4e7383d29e04233fb5b lifecycle.rs 0-Struct
https://github.com/meilisearch/meilisearch/commit/6ef73eb2262e12ad84ea7df26735954969afb342 settings.rs 2-Macro
https://github.com/nushell/nushell/commit/d9d956e54f45d5eab47359d483dbead17ebd4ed1 config.rs 1-Fn
https://github.com/nushell/nushell/commit/22469a9cb1486a07e5c0989e38e071f944f346a3 cp.rs 0-Impl
https://github.com/yewstack/yew/commit/f18eca18600b2d5f1a52aa9c35a7

In [9]:
full_dbscan_clusters.sort(key=len, reverse=True)
ID = 0
ni = 0
for cluster in full_dbscan_clusters:
    ID += 1
    if len(cluster) > 50:
        ni += 1
    print(len(cluster), "ID:", ID)
    samples = cluster.sample(min(50, len(cluster)), random_state=RANDOM_STATE)
    common_set = set()
    for index1, row in samples.iterrows():
        print("https://github.com/", end="")
        print(DS.loc[index1, 'proj'], end="/commit/")
        print(DS.loc[index1, 'commit'], end=" ")
        print(DS.loc[index1, 'file'], end=" ")
        print(DS.loc[index1, 'scope'])        
        row = row[:len(row)-1]
        row = row[row != 0]
        nonTerminals = set(row.to_dict().keys())
        print(nonTerminals)
        print("------")
        if len(common_set) == 0:
            common_set = nonTerminals
        else:
            common_set = common_set & nonTerminals
    for item in sorted(list(common_set)):
        print(item)
    print("=======\n")
print(ni)

618 ID: 1
https://github.com/firecracker-microvm/firecracker/commit/343bd06bc93c4bb6e23c86e135cf3b456fa7e3f6 ioctl.rs 0-Macro
{'Macro', 'Ident', 'ItemMacro'}
------
https://github.com/swc-project/swc/commit/2745cb79491436b1967231f905a834b06831e1b4 mod.rs 2-Macro
{'Macro', 'Ident', 'ItemMacro'}
------
https://github.com/tokio-rs/tokio/commit/7b4c999341809588a427a9a80d310ee4aa1c1a21 lib.rs nan
{'ItemMacro', 'Macro', 'Ident', 'Path', 'PathSegment'}
------
https://github.com/denoland/deno/commit/22e7b0f585fb3715ef1894b2c26a0e3cbbd43e9e bindings.rs 0-Macro
{'Macro', 'Ident', 'ItemMacro'}
------
https://github.com/swc-project/swc/commit/eb2162cbd2ab2ff84a4f21828d00dab5a29f38fe native.rs 0-Macro
{'Macro', 'Ident', 'ItemMacro'}
------
https://github.com/swc-project/swc/commit/637ef7f49f0b3b7bbb06860371e6f8b2518759c2 fixture.rs 0-Macro
{'Macro', 'Ident', 'ItemMacro'}
------
https://github.com/tokio-rs/tokio/commit/555b74c7cdc3a288b0ae16af6e6358f3e3922ca6 mod.rs 2-Macro
{'Macro', 'Ident', 'ItemM